In [ ]:
#####Switching Bi-LDA model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import gensim
import itertools
from numpy.random import *
from scipy import optimize
from scipy import sparse
import seaborn as sns

In [ ]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no):
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
    return Z

In [ ]:
####データの発生####
##データの設定
#文書の設定
k1 = 20   #ユーザーアイテム
k2 = 20   #アイテムトピック
s = 3
hh = 5000   #レビュアー数
item = 2000   #アイテム数
v1 = 500   #ユーザートピックの語彙数
v2 = 500   #アイテムトピックの語彙数
v3 = 250   #一般語の語彙数
v = v1 + v2 + v3   #総語彙数
index_v1 = np.arange(v1)
index_v2 = np.arange(v1, v1+v2)
index_v3 = np.arange(v1+v2, v)
pt = np.random.poisson(np.random.gamma(10.0, 1/0.5, hh), hh)   #ユーザーあたりの文書数
d = np.sum(pt)   #総文書数

#文章の設定
w = np.random.poisson(np.random.gamma(20.0, 1/0.25, d), d)   #文書あたりの単語数
f = np.sum(w)   #総単語数

In [ ]:
##文書のIDとインデックスを設定
#IDを設定
user_id = np.repeat(np.arange(hh), pt)
pt_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(hh)])))

#インデックスの設定
index = np.arange(d)
user_index = [i for i in range(hh)]
for i in range(hh):
    user_index[i] = index[user_id==i]

In [ ]:
##アイテムの割当を生成
#セグメント割当を生成
topic = 30
phi = np.random.dirichlet(np.repeat(0.5, item), topic)
theta = np.random.dirichlet(np.repeat(2.5, topic), hh)
z = np.dot(np.array([np.random.multinomial(1, theta[i, :], 1) for i in range(hh)]).reshape(hh, topic), range(topic))

#多項分布からアイテムを生成
item_id = np.zeros(d, dtype='int')
for i in range(hh):
    if i%1000==0:
        print(i)
    item_id[user_index[i]] = np.dot(np.random.multinomial(1, phi[z[i], :], pt[i]), range(item))

In [ ]:
##単語のIDとインデックスを設定
#IDの設定
u_id = np.repeat(user_id, w)
v_id = np.repeat(item_id, w)
d_id = np.repeat(np.arange(d), w)
t_id = np.array(list(itertools.chain(*[np.array(range(w[i]), dtype="int") for i in range(d)])))

#インデックスの設定
index = np.arange(f)
u_index = [i for i in range(hh)]
v_index = [j for j in range(item)]
for i in range(hh):
    u_index[i] = index[u_id==i]
for j in range(item):
    v_index[j] = index[v_id==j]

In [ ]:
##パラメータの事前分布の設定
#トピック分布の事前分布の設定
alpha1 = np.array([4.5, 4.0, 2.0])   #スイッチング変数の事前分布
alpha21 = np.repeat(0.2, k1)
alpha22 = np.repeat(0.15, k2)

#単語分布の事前分布の設定
beta1 = np.repeat(0.0001, v); beta2 = np.repeat(0.0001, v); beta3 = np.repeat(0.0001, v)
beta1[index_v1] = 0.025; beta2[index_v2] = 0.025; beta3[index_v3] = 1.0

In [ ]:
##すべての単語が出現するまでデータの生成を続ける
rp = 0
while True:
    rp = rp + 1
    print(rp)

    ##ディリクレ分布からパラメータを生成
    #トピック分布を生成
    Lambda = np.random.dirichlet(alpha1, hh) 
    theta1 = np.random.dirichlet(alpha21, hh)
    theta2 = np.random.dirichlet(alpha22, hh)
    Lambdat = Lambda; thetat1 = theta1; thetat2 = theta2

    #単語分布を生成
    phi = np.random.dirichlet(beta1, k1)
    gamma = np.random.dirichlet(beta2, k2)
    omega = np.random.dirichlet(beta3, 1).reshape(-1)

    #出現確率が低い単語を入れ替える
    index = np.array(range(v1))[np.max(phi[:, index_v1], axis=0) <= (k1*k2)/f]
    for j in range(index.shape[0]):
        phi[np.argmax(np.random.multinomial(1, np.repeat(1/k1, k1), 1)), index[j]] = (k1*k2)/f
    index = v1 + np.array(range(v2))[np.max(gamma[:, index_v2], axis=0) <= (k1*k2)/f]
    for j in range(index.shape[0]):
        gamma[np.argmax(np.random.multinomial(1, np.repeat(1/k2, k2), 1)), index[j]] = (k1*k2)/f
    phit = phi; gammat = gamma; omegat = omega

    ##文書ごとにデータを生成
    #データの格納用配列
    WX = np.zeros((d, v))
    wd_list = [i for i in range(d)]
    y_list = [i for i in range(d)]
    z1_list = [i for i in range(d)]
    z2_list = [i for i in range(d)]

    for i in range(d):
        #ユーザーとアイテムを抽出
        index = np.arange(w[i])
        get_user = user_id[i]
        get_item = item_id[i]

        #多項分布からスイッチグ変数を生成
        y = np.random.multinomial(1, Lambda[get_user, ], w[i])

        #ユーザートピックを生成
        z1 = np.random.multinomial(1, theta1[get_user, ], w[i]) * y[:, 0].reshape(w[i], 1)
        z1_vec = np.dot(z1, np.arange(k1))

        #アイテムトピックを生成
        z2 = np.random.multinomial(1, theta2[get_item, ], w[i]) * y[:, 1].reshape(w[i], 1)
        z2_vec = np.dot(z2, np.arange(k2))

        #トピックから単語分布を決定
        Prob = np.repeat(omega, w[i]).reshape(w[i], v, order="F")
        if np.sum(y[:, 0]) > 0:
            index_z1 = index[y[:, 0]==1]
            Prob[index_z1, ] = phi[z1_vec[index_z1], ]
        if np.sum(y[:, 1]) > 0:
            index_z2 = index[y[:, 1]==1]
            Prob[index_z2, ] = gamma[z2_vec[index_z2], ]

        #多項分布から単語を生成
        word = np.array(rmnom(Prob, w[i], v, np.arange(w[i])).todense(), dtype="int")
        word_vec = np.dot(word, np.arange(v))

        #データを格納
        WX[i, ] = np.sum(word, axis=0)
        y_list[i] = y
        z1_list[i] = z1
        z2_list[i] = z2
        wd_list[i] = word_vec
        
    #break条件
    if np.min(np.sum(WX, axis=0)) > 0:
        break

In [ ]:
##生成したデータを変換
#リストを変換
wd = np.array(list(itertools.chain(*[wd_list[i] for i in range(d)])))
y = np.array(list(itertools.chain(*[y_list[i] for i in range(d)])))
Z1 = np.array(list(itertools.chain(*[z1_list[i] for i in range(d)])))
Z2 = np.array(list(itertools.chain(*[z2_list[i] for i in range(d)])))
del wd_list; del y_list; del z1_list; del z2_list

In [ ]:
#スパース行列に変換
sparse_data = sparse.coo_matrix((np.repeat(1, f), (np.arange(f), wd)), shape=(f, v)).tocsr()
sparse_data_T = sparse_data.T
user_dt = sparse.coo_matrix((np.repeat(1, f), (u_id, np.arange(f))), shape=(hh, f)).tocsr()
item_dt = sparse.coo_matrix((np.repeat(1, f), (v_id, np.arange(f))), shape=(item, f)).tocsr()

In [ ]:
####マルコフ連鎖モンテカルロ法でSwitching Bi-LDA modelを推定####
#トピック尤度と負担率を計算する関数
def LLho(theta, phi, d_id, wd, f, k):
    Lho = theta[d_id, ] * (phi.T)[wd, ]
    topic_rate = Lho / np.sum(Lho, axis=1).reshape(f, 1)
    return Lho, topic_rate

In [ ]:
##アルゴリズムの設定
R = 2000   #サンプリング回数
keep = 2   #2回に1回の割合でサンプリング結果を格納
disp = 10
iter = 0
burnin = int(500/keep)

##事前分布の設定
#トピック分布の事前分布
alpha1 = 0.1
alpha21 = 0.1
alpha22 = 0.1

#単語分布の事前分布
beta1 = 0.0001
beta2 = 0.0001
beta3 = 0.0001

In [ ]:
##データの設定
#トピック割当確率の格納用配列
Zeros1 = np.zeros((f, k1), dtype="int") 
Zeros2 = np.zeros((f, k2), dtype="int") 

#尤度の和を計算するためのベクトル
index_f = np.arange(f)
vec_s = np.repeat(1, s)
vec_k1 = np.repeat(1, k1)
vec_k2 = np.repeat(1, k2)

In [ ]:
##パラメータの真値
#トピック分布の真値
Lambda = Lambdat
theta1 = thetat1
theta2 = thetat2

#単語分布の真値
phi = phit 
gamma = gammat
omega = omegat

In [ ]:
##パラメータの初期値
#トピック分布の初期値
np.random.dirichlet(np.repeat(2.5, s), hh)
theta1 = np.random.dirichlet(np.repeat(2.5, k1), hh)
theta2 = np.random.dirichlet(np.repeat(2.5, k2), item)

#単語分布の初期値
phi = np.random.dirichlet(np.repeat(2.5, v), k1)
gamma = np.random.dirichlet(np.repeat(2.5, v), k2)
omega = np.random.dirichlet(np.repeat(2.5, v), 1).reshape(-1)

In [ ]:
##パラメータの格納用配列
#モデルパラメータの格納用配列
LAMBDA = np.zeros((hh, s, int(R/keep)))
THETA1 = np.zeros((hh, k1, int(R/keep)))
THETA2 = np.zeros((item, k2, int(R/keep)))
PHI = np.zeros((k1, v, int(R/keep)))
GAMMA = np.zeros((k2, v, int(R/keep)))
OMEGA = np.zeros((int(R/keep), v))

#トピックの格納用配列
SEG_S = np.zeros((f, s))
SEG1 = np.zeros((f, k1))
SEG2 = np.zeros((f, k2))

In [ ]:
##対数尤度の基準値
#ユニグラムモデルの対数尤度
LLst = np.sum(np.dot(sparse_data, sparse.csr_matrix(np.log(np.sum(WX, axis=0) / f)).T).todense())
print(LLst)

#真値の対数尤度
LLbest_user = np.sum(np.log(np.dot(thetat1[u_id, ] * (phit.T)[wd, ], vec_k1)[y[:, 0]==1]))
LLbest_item = np.sum(np.log(np.dot(thetat2[v_id, ] * (gammat.T)[wd, ], vec_k2)[y[:, 1]==1]))
LLbest_normal = np.sum(np.log(omegat[wd][y[:, s-1]==1]))
LLbest = LLbest_user + LLbest_item + LLbest_normal
print(LLbest)

In [ ]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    
    ##期待尤度からスイッチング変数をサンプリング
    #トピック尤度を設定
    Lho1 = theta1[u_id, ] * (phi.T)[wd, ]
    Lho2 = theta2[v_id, ] * (gamma.T)[wd, ]
    Lho3 = omega[wd]

    #期待尤度からスイッチング確率を設定
    Lho = np.concatenate((np.dot(Lho1, vec_k1), np.dot(Lho2, vec_k2), Lho3)).reshape(f, s, order="F")
    Posterior = Lambda[u_id, ] * Lho
    Switching_Prob = Posterior / np.dot(Posterior, vec_s)[:, np.newaxis] 

    #ベルヌーイ分布からスイッチング変数をサンプリング
    Sparse_S = rmnom(Switching_Prob, f, s, np.arange(f)).tocsr()
    S = np.array(Sparse_S.todense(), dtype="int")
    s_vec = np.dot(S, np.arange(s))


    ##ディリクリ分布から混合率をサンプリング
    #ディリクリ分布のパラメータ
    ssum = np.array(np.dot(user_dt, Sparse_S).todense()) + alpha1

    #パラメータをサンプリング
    Lambda = np.zeros((hh, s))
    for i in range(hh):
        Lambda[i, ] = np.random.dirichlet(ssum[i, ], 1).reshape(-1)

    ##ユーザートピックをサンプリング
    #トピックの割当確率を設定
    index_z1 = np.where(s_vec==0)[0]
    n1 = index_z1.shape[0]
    Topic_par = Lho1[index_z1, ]
    Topic_rate = Topic_par / np.dot(Topic_par, vec_k1)[:, np.newaxis] 

    #多項分布よりトピックをサンプリング
    Zi1 = Zeros1
    Zi1[index_z1, ] = np.array(rmnom(Topic_rate, n1, k1, np.arange(n1)).todense(), dtype="int")
    z1_vec = np.dot(Zi1, np.arange(k1))
    Sparse_Zi1 = sparse.coo_matrix((np.repeat(1, n1), (index_f[index_z1], z1_vec[index_z1])), shape=(f, k1)).tocsr()


    ##アイテムトピックをサンプリング
    #トピックの割当確率を設定
    index_z2 = np.where(s_vec==1)[0]
    n2 = index_z2.shape[0]
    Topic_par = Lho2[index_z2, ]
    Topic_rate = Topic_par / np.dot(Topic_par, vec_k2)[:, np.newaxis] 

    #多項分布よりトピックをサンプリング
    Zi2 = Zeros2
    Zi2[index_z2, ] = np.array(rmnom(Topic_rate, n2, k2, np.arange(n2)).todense(), dtype="int")
    z2_vec = np.dot(Zi2, np.arange(k2))
    Sparse_Zi2 = sparse.coo_matrix((np.repeat(1, n2), (index_f[index_z2], z2_vec[index_z2])), shape=(f, k2)).tocsr()


    ##ユーザーとアイテムのトピック分布のパラメータをサンプリング
    #ディリクリ分布のパラメータ
    wsum1 = np.array(np.dot(user_dt, Sparse_Zi1).todense()) + alpha21
    wsum2 = np.array(np.dot(item_dt, Sparse_Zi2).todense()) + alpha22

    #パラメータをサンプリング
    theta1 = np.zeros((hh, k1)); theta2 = np.zeros((item, k2))
    for i in range(hh):
        theta1[i, ] = np.random.dirichlet(wsum1[i, ], 1)
    for i in range(item):
        theta2[i, ] = np.random.dirichlet(wsum2[i, ], 1)


    ##単語分布のパラメータをサンプリング
    #ディリクリ分布のパラメータ
    vsum1 = np.array(np.dot(sparse_data_T, Sparse_Zi1).todense()).T + beta1
    vsum2 = np.array(np.dot(sparse_data_T, Sparse_Zi2).todense()).T + beta2
    vsum3 = np.array(np.dot(sparse_data_T, Sparse_S[:, s-1]).todense()).reshape(-1) + beta3

    #パラメータをサンプリング
    phi = np.zeros((k1, v)); gamma = np.zeros((k2, v))
    for j in range(k1):
        phi[j, ] = np.random.dirichlet(vsum1[j, ], 1)
    for j in range(k2):
        gamma[j, ] = np.random.dirichlet(vsum2[j, ], 1)
    omega = np.random.dirichlet(vsum3, 1).reshape(-1)


    ##パラメータの格納とサンプリング結果の表示
    #サンプリング結果の格納
    if rp%keep==0:
        mkeep = rp//keep
        LAMBDA[:, :, mkeep] = Lambda
        THETA1[:, :, mkeep] = theta1
        THETA2[:, :, mkeep] = theta2
        PHI[:, :, mkeep] = phi
        GAMMA[:, :, mkeep] = gamma
        OMEGA[mkeep, :] = omega

    #トピック割当はバーンイン期間を超えたら格納
    if rp%keep==0 & rp >= burnin:
        SEG_S = SEG_S + S
        SEG1 = SEG1 + Zi1
        SEG2 = SEG2 + Zi2

    if rp%disp==0:
        #対数尤度の更新
        LL_user = np.sum(np.log(np.dot(Lho1, vec_k1)[index_z1, ]))
        LL_item = np.sum(np.log(np.dot(Lho2, vec_k2)[index_z2, ]))
        LL_normal = np.sum(np.log(Lho3[S[:, s-1]==1]))
        LL = LL_user + LL_item + LL_normal

        #サンプリング結果を確認
        print(rp)
        print(np.round(np.hstack((np.array([LL]), np.array([LLst]), np.array([LLbest]))), 1))